Que.3 -

Prepare a model for glass classification using KNN, SVM and Naive Bayes.
Also calculate Accuracy by confusion matrix, Calculate accuracy score, precision score, recall score, f1 score.
Data Description:
RI : refractive index
Na: Sodium (unit measurement: weight percent in corresponding oxide, as are attributes 4-10)
Mg: Magnesium
AI: Aluminum
Si: Silicon
K:Potassium
Ca: Calcium
Ba: Barium
Fe: Iron
Type: Type of glass: (class attribute)
1 -- building_windows_float_processed
 2 --building_windows_non_float_processed
 3 --vehicle_windows_float_processed
 4 --vehicle_windows_non_float_processed (none in this database)
 5 --containers
 6 --tableware
 7 --headlamps

Dataset : Glass.csv

### import required packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### load the data

In [2]:
df = pd.read_csv('glass.csv')
df.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


### exploratory data analysis

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RI      214 non-null    float64
 1   Na      214 non-null    float64
 2   Mg      214 non-null    float64
 3   Al      214 non-null    float64
 4   Si      214 non-null    float64
 5   K       214 non-null    float64
 6   Ca      214 non-null    float64
 7   Ba      214 non-null    float64
 8   Fe      214 non-null    float64
 9   Type    214 non-null    int64  
dtypes: float64(9), int64(1)
memory usage: 16.8 KB


In [4]:
df.isna().sum()

RI      0
Na      0
Mg      0
Al      0
Si      0
K       0
Ca      0
Ba      0
Fe      0
Type    0
dtype: int64

In [5]:
# check if the data is imbalanced
df['Type'].value_counts()

Type
2    76
1    70
7    29
3    17
5    13
6     9
Name: count, dtype: int64

In [6]:
df.cov()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
RI,9.222541e-06,-0.000476,-0.000536,-0.000618,-0.001275,-0.000574,0.003502,-5.828836e-07,0.000042,-0.001049
Na,-4.758592e-04,0.666841,-0.322422,0.063926,-0.044154,-0.141713,-0.320106,1.326109e-01,-0.019204,0.863939
Mg,-5.356087e-04,-0.322422,2.080540,-0.346967,-0.185375,0.005076,-0.910916,-3.530469e-01,0.011674,-2.260643
Al,-6.175934e-04,0.063926,-0.346967,0.249270,-0.002136,0.106138,-0.184450,1.190103e-01,-0.003620,0.628970
Si,-1.275010e-03,-0.044154,-0.185375,-0.002136,0.599921,-0.097662,-0.230085,-3.934042e-02,-0.007109,0.246967
K,-5.740478e-04,-0.141713,0.005076,0.106138,-0.097662,0.425354,-0.295006,-1.382028e-02,-0.000491,-0.013795
Ca,3.502498e-03,-0.320106,-0.910916,-0.184450,-0.230085,-0.295006,2.025366,-7.984845e-02,0.017329,0.002851
Ba,-5.828836e-07,0.132611,-0.353047,0.119010,-0.039340,-0.013820,-0.079848,2.472270e-01,-0.002844,0.601630
Fe,4.231770e-05,-0.019204,0.011674,-0.003620,-0.007109,-0.000491,0.017329,-2.843522e-03,0.009494,-0.038594
Type,-1.049273e-03,0.863939,-2.260643,0.628970,0.246967,-0.013795,0.002851,6.016300e-01,-0.038594,4.425716


In [7]:
df.corr()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
RI,1.000000,-0.191885,-0.122274,-0.407326,-0.542052,-0.289833,0.810403,-0.000386,0.143010,-0.164237
Na,-0.191885,1.000000,-0.273732,0.156794,-0.069809,-0.266087,-0.275442,0.326603,-0.241346,0.502898
Mg,-0.122274,-0.273732,1.000000,-0.481799,-0.165927,0.005396,-0.443750,-0.492262,0.083060,-0.744993
Al,-0.407326,0.156794,-0.481799,1.000000,-0.005524,0.325958,-0.259592,0.479404,-0.074402,0.598829
Si,-0.542052,-0.069809,-0.165927,-0.005524,1.000000,-0.193331,-0.208732,-0.102151,-0.094201,0.151565
K,-0.289833,-0.266087,0.005396,0.325958,-0.193331,1.000000,-0.317836,-0.042618,-0.007719,-0.010054
Ca,0.810403,-0.275442,-0.443750,-0.259592,-0.208732,-0.317836,1.000000,-0.112841,0.124968,0.000952
Ba,-0.000386,0.326603,-0.492262,0.479404,-0.102151,-0.042618,-0.112841,1.000000,-0.058692,0.575161
Fe,0.143010,-0.241346,0.083060,-0.074402,-0.094201,-0.007719,0.124968,-0.058692,1.000000,-0.188278
Type,-0.164237,0.502898,-0.744993,0.598829,0.151565,-0.010054,0.000952,0.575161,-0.188278,1.000000


### pre-processing

In [9]:
# split the data into x and y

x = df.drop(['RI','Si','K','Ca','Fe','Type'],axis=1)
y = df['Type']
x

,Na,Mg,Al,Ba
0,13.64,4.49,1.10,0.00
1,13.89,3.60,1.36,0.00
2,13.53,3.55,1.54,0.00
3,13.21,3.69,1.29,0.00
4,13.27,3.62,1.24,0.00
...,...,...,...,...
209,14.14,0.00,2.88,1.06
210,14.92,0.00,1.99,1.59
211,14.36,0.00,2.02,1.64
212,14.38,0.00,1.94,1.57


### split the data into train and test sets

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=123456)

### train the models

#### Naive Bayes

In [11]:
from sklearn.naive_bayes import GaussianNB

def create_naive_bayes_model():

    # create the model
    model = GaussianNB()

    # train the model
    model.fit(x_train, y_train)

    return model

#### KNN

In [12]:
from sklearn.neighbors import KNeighborsClassifier

def create_knn_model():

    # create the model
    model = KNeighborsClassifier(n_neighbors=5)

    # train the model
    model.fit(x_train, y_train)

    return model

#### SVM

In [13]:
from sklearn.svm import SVC

def create_svm_model():
    # create a model
    model = SVC()
    
    # train the model
    model.fit(x_train, y_train)

    return model

### model evaluation

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(model_name, model):
    
    # get the predictions using model
    y_pred = model.predict(x_test)
    y_true = y_test

    print(confusion_matrix(y_true, y_pred))
    
    # get the accuracy
    accuracy = accuracy_score(y_true, y_pred)

    # get the precision
    precision = precision_score(y_true, y_pred,average='micro')

    # get the recall
    recall = recall_score(y_true, y_pred,average='micro')

    # get f1 score
    f1 = f1_score(y_true, y_pred,average='micro')

    return model_name, accuracy, precision, recall, f1

In [21]:
# create Naive Bayes model
model_nb = create_naive_bayes_model()

# create KNN model
model_knn = create_knn_model()

# create SVM model
model_svm = create_svm_model()

In [22]:
# evaluate all the models
model_evaluation_params = pd.DataFrame([
    evaluate_model("Naive Bayes", model_nb),
    evaluate_model('KNN', model_knn), 
    evaluate_model('SVM', model_svm)
], columns=["Model Name", "Accuracy", "Precision", "Recall", "F1 Score"])

[[11  1  0  0  6  0]
 [ 4  0  0  2 17  0]
 [ 3  0  0  0  3  0]
 [ 0  0  0  1  4  0]
 [ 0  0  0  0  2  0]
 [ 0  0  0  0  2  9]]
[[15  3  0  0  0  0]
 [ 4 17  0  0  2  0]
 [ 4  2  0  0  0  0]
 [ 0  3  0  2  0  0]
 [ 0  0  0  0  2  0]
 [ 1  2  0  0  1  7]]
[[18  0  0  0  0  0]
 [18  5  0  0  0  0]
 [ 6  0  0  0  0  0]
 [ 0  3  0  0  0  2]
 [ 0  1  0  0  0  1]
 [ 2  2  0  0  0  7]]


In [23]:
# performance chart
model_evaluation_params

,Model Name,Accuracy,Precision,Recall,F1 Score
0,Naive Bayes,0.353846,0.353846,0.353846,0.353846
1,KNN,0.661538,0.661538,0.661538,0.661538
2,SVM,0.461538,0.461538,0.461538,0.461538
